<a href="https://colab.research.google.com/github/daewpark/web_scrapping/blob/main/eunpyung_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Package 설치

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

### Pactkage 호출 및 로드

In [ ]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920x1080') # 왜 윈도우 사이즈를 명시하면 에러가 안나는 거지요?
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import re

### Selenium WebDriver 구동 및 검색결과 Parsing

In [ ]:
url = "https://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001180"
driver.get(url)
time.sleep(1)

In [ ]:
# 로드 된 웹페이지에서 검색창의 Name 을 찾아서 검색어 입력
search = driver.find_element_by_name("searTot")
search.send_keys("원아모집") # headless 모드에서 chrome_options.add_argument('--window-size=1920x1080') 옵션이 없으면 에러 발생하므로 옵션을 반드시 넣을 것
search.submit()


In [ ]:
# 검색결과 파싱
driver.switch_to.window(driver.window_handles[-1]) #검색결과가 다른 탭에서 열리므로 새롭게 열린 탭으로 이동
search_html = driver.page_source
soup = BeautifulSoup(search_html,'html.parser')
title = soup.find_all(class_='btnS_nw')

### Pandas DataFrame 으로 결과저장

In [ ]:
result_list = []

for i in title :
   line = i.attrs['title'] + "   " + i.attrs['href']
   result_list.append(line)

with open ("result.txt","w",encoding="utf-8") as file :
    for i in result_list :
        file.write (i + '\n')

file.close


<function TextIOWrapper.close>

### 스크래핑 결과 메일로 보내기

In [ ]:
import smtplib;
# from email.mime.base import MIMEBase;
# 메일 메시지를 만드는 모듈 (MIMEBase는 이하 MIMEMultipart, MIMEText, MIMEApplication, MIMEImage, MIMEAudio)의 상위 모듈이다.
# 굳이 선언할 필요없다.

from email.mime.multipart import MIMEMultipart                                  # 메일의 Data 영역의 메시지를 만드는 모듈 (MIMEText, MIMEApplication, MIMEImage, MIMEAudio가 attach되면 바운더리 형식으로 변환)
from email.mime.text import MIMEText                                            # 메일의 본문 내용을 만드는 모듈
from email.mime.application import MIMEApplication                              # 메일의 첨부 파일을 base64 형식으로 변환
from email.mime.image import MIMEImage                                          # 메일의 이미지 파일을 base64 형식으로 변환(Content-ID 생성)
from email.mime.audio import MIMEAudio                                          # 메일의 음악 파일을 base64 형식으로 변환(Content-ID 생성)
import io # 파일 IO

In [ ]:
#메일 발송시간을 알리기 위하여 timestamp 라이브러리 호출
import datetime as dt
today = dt.date.today()
today_string = today.strftime("%Y-%m-%d")

# 메일 서버와 통신하기 전에 메시지를 만든다.
message = MIMEMultipart()
message.set_charset("utf-8")
message['From'] = "내 메일 주소"
message['To'] = "내 메일 주소"
message['Cc'] = None
message['Bcc'] = None
message['Subject'] = "은평구청 원아모집 공고 알림봇 (" + today_string + ")"
body = "\r\n".join(result_list)
message.attach (MIMEText(body, 'plain'))
print(message)

In [ ]:
# 첨부 파일 추가
#with open("test.xlsx", "rb") as fp :
#    part = MIMEApplication(fp.read(), Name = "test.xlsx") # Name은 메일 수신자에서 설정되는 파일 이름
#    data.attach (part) # Data 영역의 메시지에 바운더리 추가

# 이미지 파일 추가
#with open("capture.png", 'rb') as fp :
#    img = MIMEImage(fp.read(), Name = "capture.png") # Name은 메일 수신자에서 설정되는 파일 이름
#    img.add_header('Content-ID', '<capture>') # 해더에 Content-ID 추가(본문 내용에서 cid로 링크를 걸 수 있다.)
#    data.attach(img) # Data 영역의 메시지에 바운더리 추가


# Html 형식의 본문 내용 (cid로 이미 첨부 파일을 링크했다.)
# msg = MIMEText("Hello Test<br /><img src='cid:capture'>", 'html')

In [ ]:
session = smtplib.SMTP('smtp.gmail.com', 587) # 메일 서버와 통신 개시
session.set_debuglevel(True)
session.ehlo()
session.starttls()
session.login('dwn.park@gmail.com', 'xytbwmxhgsfjvune')

sender = message['From']
receiver = message['To'].split(",")
if message['Cc'] is not None :
    receiver += data['Cc'].split(",") # 참조자 추가
if message['Bcc'] is not None :
    receiver += message['Bcc'].split(",") # 숨은 참조자 추가
session.sendmail(sender, receiver, message.as_string()) # 메일 프로토콜 상 MAIL, RCPT, DATA 순으로 메시지를 보내야 하는데 이걸 sendmail함수에서 자동으로 해준다.
session.quit() # QUIT을 보내고 접속을 종료하고 메일을 보낸다.